# AUTHOR-NIKHIL MUGGANAWAR

# Predicting The Costs Of Used Cars

In [1]:
import pandas as pd
import numpy as np

In [2]:
#READ THE TRAINING DATA SET
df_train=pd.read_excel(r'C:\Users\admin\Data_Train.xlsx')

In [3]:
#UTILITY FUNCTIONS
def category_values(dataframe, categories):
    for c in categories:
        print('\n', dataframe.groupby(by=c)[c].count().sort_values(ascending=False))
        print('Nulls: ', dataframe[c].isnull().sum())

def plot_correlation_map( df ):
    corr = df.corr()
    _ , ax = plt.subplots( figsize =( 12 , 10 ) )
    cmap = sns.diverging_palette( 220 , 10 , as_cmap = True )
    _ = sns.heatmap(
        corr, 
        cmap = cmap,
        square=True, 
        cbar_kws={ 'shrink' : .9 }, 
        ax=ax, 
        annot = True, 
        annot_kws = { 'fontsize' : 12 }
    )

In [4]:
df_train.head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,CNG,Manual,First,26.6 km/kg,998 CC,58.16 bhp,5.0,NaN,1.75
1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,NaN,12.50
2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,18.2 kmpl,1199 CC,88.7 bhp,5.0,8.61 Lakh,4.50
3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,NaN,6.00
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,NaN,17.74


In [5]:
#GET THE START AND END INDEX OF TRAINING DATA SET
train_start_index=0
train_end_index=df_train.shape[0]

In [6]:
print(train_end_index)

6019


In [7]:
#READ THE TEST DATA
df_test=pd.read_excel(r'C:\Users\admin\Data_Test.xlsx')

In [8]:
df_test.head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price
0,Maruti Alto K10 LXI CNG,Delhi,2014,40929,CNG,Manual,First,32.26 km/kg,998 CC,58.2 bhp,4.0,NaN
1,Maruti Alto 800 2016-2019 LXI,Coimbatore,2013,54493,Petrol,Manual,Second,24.7 kmpl,796 CC,47.3 bhp,5.0,NaN
2,Toyota Innova Crysta Touring Sport 2.4 MT,Mumbai,2017,34000,Diesel,Manual,First,13.68 kmpl,2393 CC,147.8 bhp,7.0,25.27 Lakh
3,Toyota Etios Liva GD,Hyderabad,2012,139000,Diesel,Manual,First,23.59 kmpl,1364 CC,null bhp,5.0,NaN
4,Hyundai i20 Magna,Mumbai,2014,29000,Petrol,Manual,First,18.5 kmpl,1197 CC,82.85 bhp,5.0,NaN


In [9]:
#GET THE DEPENDENT VAIABLE SEPARATED FROM TRAINING DATA SET
df_train_Y=df_train['Price']

In [10]:
#CHECK FOR NULL VALUES IN TRAINING DATA
df_train.isnull().sum()

Name                    0
Location                0
Year                    0
Kilometers_Driven       0
Fuel_Type               0
Transmission            0
Owner_Type              0
Mileage                 2
Engine                 36
Power                  36
Seats                  42
New_Price            5195
Price                   0
dtype: int64

In [11]:
#CHECK FOR NULL VALUES IN TEST DATA COLUMN
df_test.isnull().sum()

Name                    0
Location                0
Year                    0
Kilometers_Driven       0
Fuel_Type               0
Transmission            0
Owner_Type              0
Mileage                 0
Engine                 10
Power                  10
Seats                  11
New_Price            1052
dtype: int64

In [12]:
#SINCE NUMMBER OF NULL VALUES IN NEW_PRICE COLUMN IS TOO HIGH ,WE ARE DELETING THE COLUMN IN TRAINING AND TEST DATA
df_train=df_train.drop(['New_Price','Price'],axis=1)

In [13]:
df_test=df_test.drop(['New_Price'],axis=1)

In [14]:
#GET START INDEX AND END INDEX OF TEST DATA
test_start_index=train_end_index+1
test_end_index=df_test.shape[0]+test_start_index

In [15]:
print(test_start_index)

6020


In [16]:
print(test_end_index)

7254


In [17]:
#MERGE TEST DATA AND TRAINING DATA TO CHECK FOR ALL POSSIIBLE CATEGORIES FOR TRAINING AND TEST DATA
df_inspect=pd.concat([df_train,df_test],axis=0)

In [18]:
df_inspect.head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats
0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,CNG,Manual,First,26.6 km/kg,998 CC,58.16 bhp,5.0
1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0
2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,18.2 kmpl,1199 CC,88.7 bhp,5.0
3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0


In [19]:
df_inspect.shape

(7253, 11)

In [20]:
#REPLACE NULL VALUES IN ENGINE COLUMN WITH '000 CC'
df_inspect['Engine']=df_inspect['Engine'].fillna('000 CC')

In [21]:
#REPLACE NULL VALUES IN POWER COLUMN WITH '000 bhp'
df_inspect['Power']=df_inspect['Power'].fillna('000 bhp')

In [22]:
#REPLACE NULL VALUES IN SEATS COLUMN WITH '0.0'
df_inspect['Seats']=df_inspect['Seats'].fillna(0.0)

In [23]:
#REPLACE km/kg with kmpl to standardize values in mileage column
#df['NewMileage'] = df['Mileage'].str.extract('(\d\d\d\d\.\d)', expand=True)
#df['NewMileage']
#df_inspect['Mileage']=df_inspect['Mileage'].replace('km/kg','kmpl')

In [24]:
df_inspect.head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats
0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,CNG,Manual,First,26.6 km/kg,998 CC,58.16 bhp,5.0
1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0
2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,18.2 kmpl,1199 CC,88.7 bhp,5.0
3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0


In [25]:
#REPLACE NULL VALUES IN MILEAGE COLUMN WITH '0.0 kmpl'
df_inspect['Mileage']=df_inspect['Mileage'].fillna('0.0 kmpl')

In [26]:
df_inspect.isnull().sum()

Name                 0
Location             0
Year                 0
Kilometers_Driven    0
Fuel_Type            0
Transmission         0
Owner_Type           0
Mileage              0
Engine               0
Power                0
Seats                0
dtype: int64

In [27]:
df_inspect.Engine=df_inspect.Engine.str[:-3]

In [28]:
df_inspect.head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats
0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,CNG,Manual,First,26.6 km/kg,998,58.16 bhp,5.0
1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582,126.2 bhp,5.0
2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,18.2 kmpl,1199,88.7 bhp,5.0
3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248,88.76 bhp,7.0
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968,140.8 bhp,5.0


In [29]:
df_inspect.Power=df_inspect.Power.str[:-4]

In [30]:
df_inspect.head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats
0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,CNG,Manual,First,26.6 km/kg,998,58.16,5.0
1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582,126.2,5.0
2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,18.2 kmpl,1199,88.7,5.0
3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248,88.76,7.0
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968,140.8,5.0


In [31]:
df_inspect.Mileage=df_inspect.Mileage.str[:-5]

In [32]:
df_inspect.head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats
0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,CNG,Manual,First,26.6,998,58.16,5.0
1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67,1582,126.2,5.0
2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,18.2,1199,88.7,5.0
3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77,1248,88.76,7.0
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2,1968,140.8,5.0


In [33]:
df_OwnerType=pd.get_dummies(df_inspect['Owner_Type'], drop_first=True)

In [34]:
df_inspect_OwnerType=pd.concat([df_inspect,df_OwnerType],axis=1)

In [35]:
df_inspect_OwnerType.head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Fourth & Above,Second,Third
0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,CNG,Manual,First,26.6,998,58.16,5.0,0,0,0
1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67,1582,126.2,5.0,0,0,0
2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,18.2,1199,88.7,5.0,0,0,0
3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77,1248,88.76,7.0,0,0,0
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2,1968,140.8,5.0,0,1,0


In [36]:
del df_inspect_OwnerType['Owner_Type']

In [37]:
df_TransmissionType=pd.get_dummies(df_inspect_OwnerType['Transmission'], drop_first=True)
df_inspect_TransmissionType=pd.concat([df_inspect_OwnerType,df_TransmissionType],axis=1)

In [38]:
del df_inspect_TransmissionType['Transmission']

In [39]:
df_inspect_TransmissionType.head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Mileage,Engine,Power,Seats,Fourth & Above,Second,Third,Manual
0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,CNG,26.6,998,58.16,5.0,0,0,0,1
1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,19.67,1582,126.2,5.0,0,0,0,1
2,Honda Jazz V,Chennai,2011,46000,Petrol,18.2,1199,88.7,5.0,0,0,0,1
3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,20.77,1248,88.76,7.0,0,0,0,1
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,15.2,1968,140.8,5.0,0,1,0,0


In [40]:
df_inspect_TransmissionType.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7253 entries, 0 to 1233
Data columns (total 13 columns):
Name                 7253 non-null object
Location             7253 non-null object
Year                 7253 non-null int64
Kilometers_Driven    7253 non-null int64
Fuel_Type            7253 non-null object
Mileage              7253 non-null object
Engine               7253 non-null object
Power                7253 non-null object
Seats                7253 non-null float64
Fourth & Above       7253 non-null uint8
Second               7253 non-null uint8
Third                7253 non-null uint8
Manual               7253 non-null uint8
dtypes: float64(1), int64(2), object(6), uint8(4)
memory usage: 595.0+ KB


In [41]:
df_inspect_TransmissionType['Engine']=df_inspect_TransmissionType['Engine'].astype(np.int64)

In [42]:
df_inspect_TransmissionType['Engine']=df_inspect_TransmissionType['Engine'].replace(000,df_inspect_TransmissionType['Engine'].mean()) 

In [43]:
df_inspect_TransmissionType['Mileage']=df_inspect_TransmissionType['Mileage'].astype(float)

In [44]:
df_inspect_TransmissionType['Mileage']=df_inspect_TransmissionType['Mileage'].replace(0.0,df_inspect_TransmissionType['Mileage'].mean()) 

In [45]:
df_inspect_TransmissionType.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7253 entries, 0 to 1233
Data columns (total 13 columns):
Name                 7253 non-null object
Location             7253 non-null object
Year                 7253 non-null int64
Kilometers_Driven    7253 non-null int64
Fuel_Type            7253 non-null object
Mileage              7253 non-null float64
Engine               7253 non-null float64
Power                7253 non-null object
Seats                7253 non-null float64
Fourth & Above       7253 non-null uint8
Second               7253 non-null uint8
Third                7253 non-null uint8
Manual               7253 non-null uint8
dtypes: float64(3), int64(2), object(4), uint8(4)
memory usage: 595.0+ KB


In [46]:
df_inspect_TransmissionType.head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Mileage,Engine,Power,Seats,Fourth & Above,Second,Third,Manual
0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,CNG,26.60,998.0,58.16,5.0,0,0,0,1
1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,19.67,1582.0,126.2,5.0,0,0,0,1
2,Honda Jazz V,Chennai,2011,46000,Petrol,18.20,1199.0,88.7,5.0,0,0,0,1
3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,20.77,1248.0,88.76,7.0,0,0,0,1
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,15.20,1968.0,140.8,5.0,0,1,0,0


In [47]:
df_inspect_TransmissionType.tail()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Mileage,Engine,Power,Seats,Fourth & Above,Second,Third,Manual
1229,Volkswagen Vento Diesel Trendline,Hyderabad,2011,89411,Diesel,20.54,1598.0,103.6,5.0,0,0,0,1
1230,Volkswagen Polo GT TSI,Mumbai,2015,59000,Petrol,17.21,1197.0,103.6,5.0,0,0,0,0
1231,Nissan Micra Diesel XV,Kolkata,2012,28000,Diesel,23.08,1461.0,63.1,5.0,0,0,0,1
1232,Volkswagen Polo GT TSI,Pune,2013,52262,Petrol,17.20,1197.0,103.6,5.0,0,0,1,0
1233,Mercedes-Benz E-Class 2009-2013 E 220 CDI Avan...,Kochi,2014,72443,Diesel,10.00,2148.0,170,5.0,0,0,0,0


In [48]:
df_FuelType=pd.get_dummies(df_inspect_TransmissionType['Fuel_Type'], drop_first=True)
df_inspect_FuelType=pd.concat([df_inspect_TransmissionType,df_FuelType],axis=1)

In [49]:
del df_inspect_FuelType['Fuel_Type']

In [50]:
df_inspect_FuelType.head()

,Name,Location,Year,Kilometers_Driven,Mileage,Engine,Power,Seats,Fourth & Above,Second,Third,Manual,Diesel,Electric,LPG,Petrol
0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,26.60,998.0,58.16,5.0,0,0,0,1,0,0,0,0
1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,19.67,1582.0,126.2,5.0,0,0,0,1,1,0,0,0
2,Honda Jazz V,Chennai,2011,46000,18.20,1199.0,88.7,5.0,0,0,0,1,0,0,0,1
3,Maruti Ertiga VDI,Chennai,2012,87000,20.77,1248.0,88.76,7.0,0,0,0,1,1,0,0,0
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,15.20,1968.0,140.8,5.0,0,1,0,0,1,0,0,0


In [51]:
df_Location=pd.get_dummies(df_inspect_FuelType['Location'], drop_first=True)
df_inspect_Location=pd.concat([df_inspect_FuelType,df_Location],axis=1)

In [52]:
del df_inspect_Location['Location']

In [53]:
df_inspect_Location.head()

,Name,Year,Kilometers_Driven,Mileage,Engine,Power,Seats,Fourth & Above,Second,Third,...,Bangalore,Chennai,Coimbatore,Delhi,Hyderabad,Jaipur,Kochi,Kolkata,Mumbai,Pune
0,Maruti Wagon R LXI CNG,2010,72000,26.60,998.0,58.16,5.0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,Hyundai Creta 1.6 CRDi SX Option,2015,41000,19.67,1582.0,126.2,5.0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,Honda Jazz V,2011,46000,18.20,1199.0,88.7,5.0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,Maruti Ertiga VDI,2012,87000,20.77,1248.0,88.76,7.0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,Audi A4 New 2.0 TDI Multitronic,2013,40670,15.20,1968.0,140.8,5.0,0,1,0,...,0,0,1,0,0,0,0,0,0,0


In [54]:
#df_Name=pd.get_dummies(df_inspect_Location['Name'], drop_first=True)
#df_inspect_Name=pd.concat([df_inspect_Location,df_Name],axis=1)
df_inspect_Name=df_inspect_Location

In [55]:
#DELETING NAME COLUMN SINCE THERE ARE TOO ANY COLUMNS TO CATEGORIZE
del df_inspect_Name['Name']

In [56]:
df_inspect_Name.head()

,Year,Kilometers_Driven,Mileage,Engine,Power,Seats,Fourth & Above,Second,Third,Manual,...,Bangalore,Chennai,Coimbatore,Delhi,Hyderabad,Jaipur,Kochi,Kolkata,Mumbai,Pune
0,2010,72000,26.60,998.0,58.16,5.0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
1,2015,41000,19.67,1582.0,126.2,5.0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
2,2011,46000,18.20,1199.0,88.7,5.0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0
3,2012,87000,20.77,1248.0,88.76,7.0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0
4,2013,40670,15.20,1968.0,140.8,5.0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0


In [57]:
df_inspect_Name_train=df_inspect_Name[0:train_end_index]

In [58]:
df_inspect_Name_train.shape

(6019, 24)

In [59]:
df_trainset=pd.concat([df_inspect_Name_train,df_train_Y],axis=1)

In [60]:
df_trainset.head()

,Year,Kilometers_Driven,Mileage,Engine,Power,Seats,Fourth & Above,Second,Third,Manual,...,Chennai,Coimbatore,Delhi,Hyderabad,Jaipur,Kochi,Kolkata,Mumbai,Pune,Price
0,2010,72000,26.60,998.0,58.16,5.0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,1.75
1,2015,41000,19.67,1582.0,126.2,5.0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,12.50
2,2011,46000,18.20,1199.0,88.7,5.0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,4.50
3,2012,87000,20.77,1248.0,88.76,7.0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,6.00
4,2013,40670,15.20,1968.0,140.8,5.0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,17.74


In [61]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.metrics import mean_squared_error

from sklearn import ensemble

In [62]:
df_inspect_Name_train.dtypes

Year                   int64
Kilometers_Driven      int64
Mileage              float64
Engine               float64
Power                 object
Seats                float64
Fourth & Above         uint8
Second                 uint8
Third                  uint8
Manual                 uint8
Diesel                 uint8
Electric               uint8
LPG                    uint8
Petrol                 uint8
Bangalore              uint8
Chennai                uint8
Coimbatore             uint8
Delhi                  uint8
Hyderabad              uint8
Jaipur                 uint8
Kochi                  uint8
Kolkata                uint8
Mumbai                 uint8
Pune                   uint8
dtype: object

In [63]:
#df_inspect_Name_train['Power']=df_inspect_Name_train['Power'].fillna(0)
#df_inspect_Name_train['Power']=df_inspect_Name_train['Power'].replace('000',df_inspect_Name_train['Power'].mean())
df_inspect_Name_train['Seats']=df_inspect_Name_train['Seats'].replace('0.0',df_inspect_Name_train['Seats'].mean())

C:\PGDML\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [64]:
df_inspect_Name_train.head()

,Year,Kilometers_Driven,Mileage,Engine,Power,Seats,Fourth & Above,Second,Third,Manual,...,Bangalore,Chennai,Coimbatore,Delhi,Hyderabad,Jaipur,Kochi,Kolkata,Mumbai,Pune
0,2010,72000,26.60,998.0,58.16,5.0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
1,2015,41000,19.67,1582.0,126.2,5.0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
2,2011,46000,18.20,1199.0,88.7,5.0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0
3,2012,87000,20.77,1248.0,88.76,7.0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0
4,2013,40670,15.20,1968.0,140.8,5.0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0


In [65]:
# df_inspect_Name_train.Power.replace('[A-z]+','',regex=True)

In [66]:
del df_inspect_Name_train['Power']

In [67]:
# from sklearn.tree import DecisionTreeRegressor

In [68]:
# model = DecisionTreeRegressor() 

In [69]:
# np.where(pd.isnull(df_inspect_Name_train))

In [70]:
#df_subset.head()

In [71]:
#df_inspect_Name_train['Power']=df_inspect_Name_train['Power'].astype(float)

In [72]:
#df_inspect_Name_train['Power']=df_inspect_Name_train['Power'].replace('000',df_inspect_Name_train['Power'].mean())
#df_inspect_Name_train['Power']=df_inspect_Name_train['Power'].replace('null',df_inspect_Name_train['Power'].mean())

In [73]:
# model.fit(df_inspect_Name_train, df_train_Y)
# model.score(df_inspect_Name_train, df_train_Y)

In [74]:
df_inspect_Name_test=df_inspect_Name[test_start_index:test_end_index]

In [75]:
# import re
# #re.search([A-z]+,df_inspect_Name_test['Power'])
# re.search(' ',df_inspect_Name_test['Power'])

In [76]:
# df_inspect_Name_test.Power.replace('[A-z]+','',regex=True)

In [77]:
del df_inspect_Name_test['Power']

In [78]:
# model.predict(df_inspect_Name_test)

In [79]:
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
# model with the best hyperparameters
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor

In [80]:
# # Create the parameter grid based on the results of random search 
# param_grid = {
#     'max_depth': [4,8,10],
#     'min_samples_leaf': range(100, 400, 200),
#     'min_samples_split': range(200, 500, 200),
#     'n_estimators': [100,200, 300], 
#     'max_features': [5, 10]
# }
# # Create a based model
# rf = RandomForestRegressor()
# # Instantiate the grid search model
# grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
#                           cv = 3, n_jobs = -1,verbose = 1)

In [81]:
# # Fit the grid search to the data
# grid_search.fit(df_inspect_Name_train, df_train_Y)

In [82]:
# # printing the optimal accuracy score and hyperparameters
# print('We can get accuracy of',grid_search.best_score_,'using',grid_search.best_params_)

In [83]:
# # model with the best hyperparameters
# from sklearn.ensemble import RandomForestRegressor
# rfc = RandomForestRegressor(bootstrap=True,
#                              max_depth=8,
#                              min_samples_leaf=100, 
#                              min_samples_split=200,
#                              max_features=10,
#                              n_estimators=200)

In [84]:
# rfc.fit(df_inspect_Name_train, df_train_Y)

In [85]:
# rfc.score(df_inspect_Name_train, df_train_Y)

In [86]:
# predictions=rfc.predict(df_inspect_Name_test)

In [87]:
params = {'n_estimators': 500, 'max_depth': 8, 'min_samples_split': 200,
          'learning_rate': 0.01, 'loss': 'ls'}
clf = ensemble.GradientBoostingRegressor(**params)

In [88]:
clf.fit(df_inspect_Name_train, df_train_Y)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.01, loss='ls', max_depth=8, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=200, min_weight_fraction_leaf=0.0,
             n_estimators=500, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)

In [89]:
clf.score(df_inspect_Name_train, df_train_Y)

0.9142188038758453

In [90]:
test_train=clf.predict(df_inspect_Name_train)

In [91]:
from sklearn import metrics
print('R Score :',metrics.r2_score(df_train_Y, test_train))

R Score : 0.9142188038758453


In [92]:
pred_ran_test=clf.predict(df_inspect_Name_test)
pred_ran_test=pred_ran_test.astype(float).round(2)
print(pred_ran_test)

[ 3.22 16.87  4.03 ...  3.54  4.51 19.25]


In [93]:
df_result = pd.DataFrame (pred_ran_test)

In [94]:
#filepath = 'CarPricePrediction_08JUNE2019_13:08.xlsx'
#df.to_excel(filepath, index=False)